In [ ]:
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap
import matplotlib.image as mpimg

def rect_to_gaussian_params(rect):
    """
    Convert rectangle to Gaussian parameters.
    rect: tuple (x1, y1, x2, y2)
    Returns: mean and covariance matrix
    """
    x1, y1, x2, y2 = rect
    width = x2 - x1
    height = y2 - y1
    mean = [(x1 + x2) / 2, (y1 + y2) / 2]
    area = width * height
    variance = 1 / 5*area  # Inversely proportional to the area
    cov = [[7*variance * width**2, 0], [0, 7*variance * height**2]]
    return mean, cov

# Example rectangle coordinates with values between 0 and 1
rectangles = [
    (0.1, 0.1, 0.3, 0.4),
    (0.5, 0.5, 0.7, 0.8),
    (0.2, 0.5, 0.4, 0.6)
]

# Generate sample data from these rectangles
np.random.seed(0)
X = np.vstack([
    np.clip(np.random.multivariate_normal(*rect_to_gaussian_params(rect),20), 0, 1)
    for rect in rectangles
])

# Fit the Gaussian Mixture Model
means = np.array([rect_to_gaussian_params(rect)[0] for rect in rectangles])
covariances = np.array([rect_to_gaussian_params(rect)[1] for rect in rectangles])
weights = np.array([1.0 / (rect[2] - rect[0]) / (rect[3] - rect[1]) for rect in rectangles])
weights /= weights.sum()  # Normalize to sum to 1

gmm = GaussianMixture(n_components=len(rectangles), covariance_type='full')
gmm.weights_ = weights
gmm.means_ = means
gmm.covariances_ = covariances
gmm.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covariances))

# Predict the regions
labels = gmm.predict(X)

img = mpimg.imread(r'C:\Users\dell\Documents\space\lake.png')

fig, ax = plt.subplots()
ax.imshow(img, extent=[0, 1, 0, 1])  # Assuming the image covers the [0, 1] x [0, 1] area
cmap = ListedColormap(['r', 'g', 'b'])

# Scatter plot of the data points
scatter = ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap=cmap, edgecolors='k')

# Draw the rectangles
for rect in rectangles:
    x1, y1, x2, y2 = rect
    width = x2 - x1
    height = y2 - y1
    rect_patch = patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='w', facecolor='none')
    ax.add_patch(rect_patch)

plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title('Gaussian Mixture Model Clustering with Background Image')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()


In [38]:
import json

with open(r'C:\Users\dell\Documents\space\pred_process_no_ft.json', 'r') as file:
    pred_process = json.load(file)
with open(r'C:\Users\dell\Documents\space\region_list.json', 'r') as file:
    region_list = json.load(file)
    
for item in pred_process:
    [x1,y1,x2,y2] = item['gt_r']
    item['gt_r'] = (x1,y1,x2,y2)

In [41]:
proposals_list = []
proposals = []
for regions in region_list:
    # print(item)
    if len(regions)==0:
        proposals_list.append('null')
        print('null')
        continue
    proposals = []
    for item in regions:
        [[x1,y1],[x2,y2]] = item
        proposal = (x1,y1,x2,y2)
        proposals.append(proposal)
    proposals_list.append(proposals)

print(proposals_list[0])


null
null
null
null
null
null
[(0.6418652057647705, 0.46151906590172453, 0.7853203773498535, 0.9416204105384662), (0.007582515478134155, 0.6120208919398239, 0.22777259349822998, 0.7805214600087146)]


In [43]:
def calculate_iou(rect1, rect2):
    # rect1 and rect2 are tuples in the form (x1, y1, x2, y2)

    # Calculate intersection coordinates
    intersection_x1 = max(rect1[0], rect2[0])
    intersection_y1 = max(rect1[1], rect2[1])
    intersection_x2 = min(rect1[2], rect2[2])
    intersection_y2 = min(rect1[3], rect2[3])

    # Check if intersection exists
    if intersection_x1 > intersection_x2 or intersection_y1 > intersection_y2:
        return 0.0

    # Calculate intersection area
    intersection_area = max(0, intersection_x2 - intersection_x1) * max(0, intersection_y2 - intersection_y1)

    # Calculate union area
    area_rect1 = (rect1[2] - rect1[0]) * (rect1[3] - rect1[1])
    area_rect2 = (rect2[2] - rect2[0]) * (rect2[3] - rect2[1])
    union_area = area_rect1 + area_rect2 - intersection_area

    # Calculate IOU
    iou = intersection_area / union_area

    return iou

In [45]:
def rect_to_gaussian_params(rect):
    """
    Convert rectangle to Gaussian parameters.
    rect: tuple (x1, y1, x2, y2)
    Returns: mean and covariance matrix
    """
    x1, y1, x2, y2 = rect
    width = x2 - x1
    height = y2 - y1
    mean = [(x1 + x2) / 2, (y1 + y2) / 2]
    area = width * height
    variance = 1 / 10*area  # Inversely proportional to the area
    cov = [[10*variance * width**2, 0], [0,10*variance * height**2]]
    return mean, cov

In [46]:
count = 0
null_pred_count = 0
null_prop_count = 0
for index in range(3422):
    if proposals_list[index]=='null':
        # print('null_proposal')
        null_prop_count +=1
        continue
    rectangles = proposals_list[index]

    # Generate sample data from these rectangles
    np.random.seed(0)
    if len(pred_process[index]['pred_points'])==0:
        # print('null_pred')
        null_pred_count +=1
        continue
    X = np.vstack(pred_process[index]['pred_points'])

    gt = pred_process[index]['gt_r']


    # Fit the Gaussian Mixture Model
    means = np.array([rect_to_gaussian_params(rect)[0] for rect in rectangles])
    covariances = np.array([rect_to_gaussian_params(rect)[1] for rect in rectangles])
    weights = np.array([1.0 / (rect[2] - rect[0]) / (rect[3] - rect[1]) for rect in rectangles])
    weights /= weights.sum()  # Normalize to sum to 1

    gmm = GaussianMixture(n_components=len(rectangles), covariance_type='full')
    gmm.weights_ = weights
    gmm.means_ = means
    gmm.covariances_ = covariances
    gmm.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covariances))

    # Predict the regions
    labels = gmm.predict(X)

    prediction = np.argmax(np.bincount(labels))

    iou = calculate_iou(rectangles[prediction],gt)
    if iou>0.5:
        count +=1

print('match:',count/3422)
print('null pred: ', null_pred_count)
print('null proposal',null_prop_count)




match: 0.8889538281706605
null pred:  128
null proposal 6
